Alineador y traductor de ADN-ARN final

In [4]:
import sys  # Se utiliza para interactuar con el intérprete Python
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QLineEdit, QPushButton, QVBoxLayout, QRadioButton, QMessageBox
# Importamos los módulos necesarios de PyQt5
from PyQt5.QtCore import Qt  # Importamos Qt del módulo QtCore

# Definimos la función de alineación Smith-Waterman
def smith_waterman(seq1, seq2):
    # Determinamos la longitud de las secuencias
    m = len(seq1)  # Longitud de la secuencia 1
    n = len(seq2)  # Longitud de la secuencia 2

    # Creamos una matriz llena de ceros de tamaño (m+1)x(n+1)
    scores = [[0 for _ in range(n+1)] for _ in range(m+1)]
    # El puntaje máximo se inicializa en 0
    max_score = 0

    # Iteramos a través de las secuencias
    for i in range(1, m+1):
        for j in range(1, n+1):
            # Comparamos los elementos de las secuencias y sumamos al puntaje de la diagonal anterior si coinciden
            match = scores[i-1][j-1] + (seq1[i-1] == seq2[j-1])
            # Restamos uno al puntaje de la celda anterior en la misma columna
            delete = scores[i-1][j] - 1
            # Restamos uno al puntaje de la celda anterior en la misma fila
            insert = scores[i][j-1] - 1
            # El nuevo puntaje es el máximo de los tres calculados
            scores[i][j] = max(0, match, delete, insert)
            # Si el nuevo puntaje es mayor que el puntaje máximo, se actualiza
            max_score = max(max_score, scores[i][j])

    # La función devuelve el puntaje máximo
    return max_score

# Definimos la función para convertir una secuencia de ADN a ARN
def dna_to_rna(dna_sequence):
    # Reemplazamos todas las 'T' por 'U'
    return dna_sequence.replace("T", "U")

# Definimos la clase de la ventana principal
class MainWindow(QWidget):
    def __init__(self, parent=None):
        super(MainWindow, self).__init__(parent)  # Llamamos al constructor de QWidget
        self.setWindowTitle("Alineador y Traductor de Secuencias de ADN")  # Establecemos el título de la ventana

        # Creamos los widgets para las secuencias
        self.label_seq1 = QLabel("Sequence 1")  # Etiqueta para la primera secuencia
        self.line_edit_seq1 = QLineEdit()  # Campo de entrada para la primera secuencia
        self.label_seq2 = QLabel("Sequence 2")  # Etiqueta para la segunda secuencia
        self.line_edit_seq2 = QLineEdit()  # Campo de entrada para la segunda secuencia

        # Creamos el botón para proceder con la operación seleccionada
        self.button_proceed = QPushButton("Proceed")
        # Conectamos el evento del click del botón con la función de proceder
        self.button_proceed.clicked.connect(self.proceed)

        # Creamos los botones de radio para seleccionar la operación
        self.align_radio = QRadioButton("Align Sequences")  # Botón para seleccionar la alineación
        # Conectamos el evento de cambio del botón de radio con la función on_toggle
        self.align_radio.toggled.connect(self.on_toggle)
        self.translate_radio = QRadioButton("Translate Sequence")  # Botón para seleccionar la traducción
        # Conectamos el evento de cambio del botón de radio con la función on_toggle
        self.translate_radio.toggled.connect(self.on_toggle)

# Añadimos los widgets a un layout vertical
# Crear un objeto QVBoxLayout, que es una clase que proporciona
# una caja vertical para organizar widgets de manera vertical.

        layout = QVBoxLayout()
# Agregar el botón de radio para seleccionar la alineación
# debajo del campo de texto de la segunda secuencia.
        layout.addWidget(self.align_radio)
# Agregar el botón de radio para seleccionar la traducción
# debajo del botón de radio de alineación.
        layout.addWidget(self.translate_radio)
# Agregar el primer widget (etiqueta de la primera secuencia) al layout vertical.
# Esto significa que este widget se colocará en la parte superior del layout.
        layout.addWidget(self.label_seq1)
        # Agregar el segundo widget (campo de texto para la entrada de la primera secuencia)
        # debajo del primer widget en el layout vertical.
        layout.addWidget(self.line_edit_seq1)
        # Agregar la etiqueta de la segunda secuencia debajo del campo de texto de la primera secuencia.
        layout.addWidget(self.label_seq2)
        # Agregar el campo de texto para la entrada de la segunda secuencia
        # debajo de la etiqueta de la segunda secuencia.
        layout.addWidget(self.line_edit_seq2)
        # Agregar el botón de proceder debajo del botón de radio de traducción.
        layout.addWidget(self.button_proceed)


        # Establecemos el layout de la ventana
        self.setLayout(layout) # Función que se ejecuta cuando se pulsa el botón de proceder

    # Esta función se llama cuando cambia el estado de los botones de radio
    def on_toggle(self):
        # Obtenemos el botón de radio que generó el evento
        radio_button = self.sender()
        # Comprobamos si el botón de radio está seleccionado
        if radio_button.isChecked():
            # Si el botón de alineación está seleccionado, mostramos la entrada de la segunda secuencia
            if radio_button == self.align_radio:
                self.label_seq2.show()
                self.line_edit_seq2.show()
            # Si el botón de traducción está seleccionado, ocultamos la entrada de la segunda secuencia
            else:
                self.label_seq2.hide()
                self.line_edit_seq2.hide()

    # Esta función se llama cuando se pulsa el botón de proceder
    def proceed(self):
        # Si el botón de alineación está seleccionado
        if self.align_radio.isChecked():
            # Obtenemos las secuencias de las entradas
            seq1 = self.line_edit_seq1.text()
            seq2 = self.line_edit_seq2.text()
            # Si alguna de las secuencias está vacía, mostramos un mensaje de error
            if not seq1 or not seq2:
                QMessageBox.warning(self, "Warning", "Please enter both sequences")
                return
            # Si ambas secuencias están presentes, procedemos con la alineación
            score = smith_waterman(seq1, seq2)
            # Mostramos el puntaje de la alineación
            QMessageBox.information(self, "Alignment Score", f"Alignment Score: {score}")
        # Si el botón de traducción está seleccionado
        elif self.translate_radio.isChecked():
            # Obtenemos la secuencia de la entrada
            seq1 = self.line_edit_seq1.text()
            # Si la secuencia está vacía, mostramos un mensaje de error
            if not seq1:
                QMessageBox.warning(self, "Warning", "Please enter a sequence")
                return
            # Si la secuencia está presente, procedemos con la traducción
            rna_seq = dna_to_rna(seq1)
            # Mostramos la secuencia de ARN
            QMessageBox.information(self, "RNA Sequence", f"RNA Sequence: {rna_seq}")

# Esta es la función principal que ejecuta la aplicación
def main():
    # Creamos una instancia de la aplicación
    app = QApplication(sys.argv)
    # Creamos una instancia de la ventana principal
    window = MainWindow()
    # Mostramos la ventana
    window.show()
    # Ejecutamos la aplicación
    sys.exit(app.exec_())

# Si el script se está ejecutando directamente, ejecutamos la función principal
if __name__ == "__main__":
    main()


SystemExit: 0